In [1]:
%cd src

/home/student/Documents/evaluation/rke_guidance/sparke-diffusers/src


In [1]:
from diffusers import SPARKEGuidedStableDiffusionXLPipeline

In [4]:
%cd sparke_diffusers

/home/student/Documents/evaluation/rke_guidance/sparke-diffusers/sparke_diffusers


In [1]:
pip install -e .

Obtaining file:///home/student/Documents/evaluation/rke_guidance/sparke-diffusers
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for diffusers (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.33.0.dev0-0.editable-py3-none-any.whl size=8692 sha256=93723679d89520c9fe4db7a3a62c8e01a408b8ca14ce4d45b4b4b6a14ab78fd0
  Stored in directory: /tmp/pip-ephem-wheel-cache-t4ad49h4/wheels/09/67/63/03196833542794d2fd26f1995fb2a0f64de40e3e84ec8e46bc
Successfully built diffusers
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.33.0.dev0
    Uninstalling diffusers-0.33.0.dev0:
      Successfully uninstalled diffusers-0.33.0.dev0
Note: you may need to restart the kernel to use updated packages.


In [5]:
import logging
import clip

from diffusers.pipelines.rke_guidance_utils import RKEGuidedSampling


logging.basicConfig(
    level=logging.INFO,  # or logging.DEBUG
    format='[%(asctime)s] [%(levelname)s] %(message)s',
    datefmt='%H:%M:%S'
)
logger = logging.getLogger(__name__)

clip_for_guidance, pre_process_clip = clip.load("ViT-B/32", device='cuda')

# Stable Diffusion + SPARKE Diversity Guidance

For sampling using SD-1.5 and SD-2.1 use the following code

In [3]:
from diffusers import SPARKEGuidedStableDiffusionPipeline
import torch

pipe = SPARKEGuidedStableDiffusionPipeline.from_pretrained(
    "/media/student/data/models/SD2.1/stable-diffusion-2-1/",  # Replace with your actual model path
    torch_dtype=torch.float16,
).to("cuda")

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [11]:
rke_guided_sampler = RKEGuidedSampling(
    algorithm="cond-rke",
    kernel="gaussian",
    sigma_image=0.6,
    sigma_text=0.3,
    max_bank_size=1000,
    use_latents_for_guidance=True,
    model_name="sdv2.1",
    model=pipe
)

In [17]:
guidance_kwargs = dict(
    guidance_scale=7.5,
    criteria="vscore_clip",
    algorithm="vscore_clip",
    criteria_guidance_scale=0.02,
    # use_latents_for_guidance=True,
    # kernel="gaussian",
    # sigma_image=0.6,
    # sigma_text=0.3,
    guidance_freq=10,
    F_M=None,
    F_T=None,
    F_M_real=None,
    F_T_real=None,
    beta=0,
    regions_list=[],
    return_kernels=False,
    rke_guided_sampler=rke_guided_sampler,
    logger_=logger,
    clip_for_guidance=clip_for_guidance,
    regularize=False,
    regularize_weight=0,
)

prompts = [
    "A banana",
    "A happy asian woman",
    "A happy asian woman",
    # "A futuristic cityscape at sunset"
    # "A photorealistic portrait of an astronaut in space",
    # "A cyberpunk street market at night with glowing signs"
]

out = None
# Generate and save images
for i, prompt in enumerate(prompts):
    out = pipe(
        prompt=prompt + "4K, realistic",
        negative_prompt="blurry, low-quality, distorted",
        generator=torch.Generator(device="cuda").manual_seed(0),
        num_inference_steps=51,
        **guidance_kwargs
    )
    image = out[0][0]
    image.save(f"sparke_sdv2.1_output_{i}.png")


  0%|          | 0/51 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]

# Stable Diffusion XL + SPARKE Diversity Guidance

In [3]:
from diffusers import SPARKEGuidedStableDiffusionXLPipeline
import torch


pipe = SPARKEGuidedStableDiffusionXLPipeline.from_pretrained(
    "/media/student/data/models/SDXL.1/stable-diffusion-xl-base-1.0/",  # Replace with your actual model path
    torch_dtype=torch.float16,
).to("cuda")



Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [45]:
rke_guided_sampler = RKEGuidedSampling(
    algorithm="cond-rke",
    kernel="gaussian",
    sigma_image=0.6,
    sigma_text=0.3,
    max_bank_size=1000,
    use_latents_for_guidance=True,
    model_name="sdxl",
    model=pipe
)

In [ ]:
guidance_kwargs = dict(
    guidance_scale=7.5,
    criteria="vscore_clip",
    algorithm="vscore_clip",
    criteria_guidance_scale=0.025,
    use_latents_for_guidance=True,
    kernel="gaussian",
    # sigma_image=0.6,
    # sigma_text=0.3,
    guidance_freq=10,
    F_M=None,
    F_T=None,
    F_M_real=None,
    F_T_real=None,
    beta=0,
    regions_list=[],
    return_kernels=False,
    rke_guided_sampler=rke_guided_sampler,
    logger_=logger,
    clip_for_guidance=clip_for_guidance,
    regularize=False,
    regularize_weight=0,
)

prompts = [
    "A happy asian woman",
    "A happy asian woman",
    "A happy asian woman",
    # "A futuristic cityscape at sunset"
    # "A photorealistic portrait of an astronaut in space",
    # "A cyberpunk street market at night with glowing signs"
]

out = None
# Generate and save images
for i, prompt in enumerate(prompts):
    out = pipe(
        prompt=prompt + "4K, realistic",
        negative_prompt="blurry, low-quality, distorted",
        generator=torch.Generator(device="cuda").manual_seed(0),
        num_inference_steps=51,
        **guidance_kwargs
    )
    image = out[0][0]
    image.save(f"sparke_output_{i}.png")


  0%|          | 0/51 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]

# PixArt-$\Sigma$ + SPARKE Diversity Guidance

In [1]:
from diffusers import SPARKEGuidedPixArtSigmaPipeline